In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [2]:
def contains_number(string):
    return any(char.isdigit() for char in string)

# Text list

In [3]:
### Extract the html webpage
all_pages = []

for i in range(0,25) :
    try :
        url = "https://home.treasury.gov/news/press-releases/statements-remarks/secretary?title=&publication-start-date=&publication-end-date=&page=" + str(i)
        user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
        headers = {'User-Agent': user_agent}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser') 

        all_url = []
        url_list = soup.find_all('a', href=True)
        for i in url_list :
            if "/news/press-releases/" in i.get("href") :
                all_url.append(i.get("href"))
        all_pages = all_pages + all_url
    except: 
        print("exception page ", str(i))

In [4]:
all_pages_clean = []
for i in all_pages :
    if contains_number(i) == True:
        all_pages_clean.append(i)

In [5]:
def contains_number(string):
    return any(char.isdigit() for char in string)

# Text and dates extraction

In [6]:
def get_title(soup) :
    title = soup.find_all("h2", class_ = None)
    for k in title :
        if k.find("span") != None :
            text = k.find("span")
            text = text.text
    return text

def get_text(soup) :
    text_list = soup.find_all("p", class_= None)
    text = ""
    for i in text_list :
        if len(i.text) > 150 :
            text = text + i.text
    return text

def get_date(soup) :
    date = soup.find_all("time")
    return date[-1].text

In [7]:
all_title = []
all_text = []
all_date = []

for i in tqdm(all_pages_clean) :
    domain = "https://home.treasury.gov" 
    url = domain + i
    user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser') 
    
    try :
        all_title.append(get_title(soup))
    except :
        all_title.append("")
        
    try :
        all_text.append(get_text(soup))
    except :
        all_text.append("")
        
    try :
        all_date.append(get_date(soup))
    except :
        all_date.append("")

100%|█████████████████████████████████████████| 400/400 [02:35<00:00,  2.57it/s]


In [8]:
data = pd.DataFrame(data = {
    "title" : all_title, 
    "date" : all_date, 
    "text" : all_text
})
data.to_csv("/Users/flav/Documents/Cours/BigData2_BDSE/speeches.csv")

In [9]:
data

,title,date,text
0,,"November 27, 2023","WASHINGTON – From November 26 – December 1, Un..."
1,,"November 22, 2023",WASHINGTON – The U.S. Department of the Treasu...
2,,"November 21, 2023",WASHINGTON — The U.S. Department of the Treasu...
3,,"November 21, 2023",
4,,"November 17, 2023","Good afternoon. My name is Graham Steele, and ..."
...,...,...,...
395,,"December 1, 2020","Chairwoman Waters, Ranking Member McHenry, and..."
396,,"November 30, 2020",Good evening. It is my distinct honor to be he...
397,,"October 29, 2020",WASHINGTON – U.S. Treasury Secretary Steven T....
398,,"October 14, 2020",WASHINGTON – As we begin the Annual Meetings o...
